## **Import Packages and Set Starting Parameters**

In [138]:
import pandas as pd
import os
from pathlib import Path

# get current working directory
bp = Path(os.getcwd())

# set results directory
results_path = bp / "results"

In [139]:
# Ask for county to gather data for.
county = input('Enter county: ')

In [140]:
# List of contaminants from CES Drinking Water Quality index plux BTEX and MTBE.
contaminants_2 = [
    'AS',
    'BZ',
    'BZME',
    'CD',
    'CR6',
    'DBCP',
    'EBZ',
    'EDB',
    'HAA5',
    'NO3N',
    'MTBE',
    'PB',
    'PCATE',
    'PCE',
    'TCE',
    'TCPR123',
    'THM',
    'XYLENES',
    ]

# Selected contaminants
contaminants_3 = [
    'BZ', 
    'BZME', 
    'DBCP', 
    'EBZ', 
    'EDB',  
    'MTBE', 
    'PCE', 
    'TCE', 
    'TCPR123', 
    'XYLENES', 
    ]


## **Open and Concat Data**

### Open Table Function

In [141]:
# Function to open data file.
"""
open_table() is a function that opens a csv file and returns a dataframe. 
Will try to open the file with the default encoding, if that fails, will try with the unicode_escape encoding.

---------------------------------------------------------------------------------------------------------------------
Args:
    p: path to file
    dtypes: dictionary of data types
    date_cols: list of columns to parse as dates
    cols: list of columns to use
"""

def open_table(p, dtypes, date_cols, cols):

    try:
        df = pd.read_csv(p, sep='\t', dtype=dtypes, on_bad_lines='warn', parse_dates=date_cols, usecols=cols)
        return df
        
    except:
        try:
            df = pd.read_csv(p, sep='\t', dtype=dtypes, on_bad_lines='warn', encoding='unicode_escape', parse_dates=date_cols, usecols=cols)
            return df
            
        except:
            df = pd.read_csv(p, sep='\t', dtype=dtypes, engine='python', encoding='unicode_escape', on_bad_lines='warn', parse_dates=date_cols, usecols=cols)
            return df

### Sample Data

In [142]:
# Create geotracker path.
edf_path = bp / 'geotracker_edf_results'

# Dictionary of data types for geotracker edf_results for open_table().
geotracker_dtypes = {
    'GLOBAL_ID' : 'string',
    'FIELD_PT_NAME' : 'string',
    'PARLABEL' : 'string',
    'PARVAL' : 'Float64',
    'PARVQ' : 'string',
    'REPDL' : 'Float64',
    'UNITS' : 'string',
    }

# Date column of geotracker edf_results for open_table().
geotracker_date = ['LOGDATE']

# Columns of geotracker edf_results for open_table().
geotracker_cols = list(geotracker_dtypes.keys()) + geotracker_date

print('Loading Geotracker EDF results: \n')

# create list of files to open
edf_files = edf_path.glob('**/*{}*.zip'.format(county))

# Use list comprehension to create a list of dataframes from the files list. Uses open_table() to open the files.
edf_results_list = [open_table(i,geotracker_dtypes,geotracker_date,geotracker_cols) for i in edf_files]

# Concatenate the list of dataframes into one dataframe if there are more than one.
print('Concatenating EDF results: \n')
if len(edf_results_list) > 1:
    edf_results = pd.concat(edf_results_list)

else:
    edf_results = edf_results_list[0]

# Create WID column.
edf_results['WID'] = edf_results['GLOBAL_ID'] + '-' + edf_results['FIELD_PT_NAME']
edf_results['WID'] = edf_results['WID'].str.replace(' ','')

# Drop unnecessary columns.
edf_results = edf_results.drop(columns=['GLOBAL_ID', 'FIELD_PT_NAME'])

Loading Geotracker EDF results: 

Concatenating EDF results: 



In [143]:
# Set path of gama_results.
gama_path = bp / 'gama_results'

# Dictionary of data types for gama_results for open_table().
gama_dtypes = {
    'GM_WELL_ID' : 'string',
    'GM_CHEMICAL_VVL' : 'string',
    'GM_RESULT_MODIFIER' : 'string',
    'GM_RESULT' : 'Float64',
    'GM_RESULT_UNITS' : 'string',
    'GM_REPORTING_LIMIT' : 'Float64',
    }

# Date column of gama_results for open_table().
gama_date = ['GM_SAMP_COLLECTION_DATE']

# Columns of gama_results for open_table().
gama_cols = list(gama_dtypes.keys()) + gama_date

print('Loading GAMA results: \n')

# Create list of files to open.
gama_files = gama_path.glob('**/*{}*.zip'.format(county.lower()))

# Use list comprehension to create a list of dataframes from the files list. Uses open_table() to open the files.

gama_results_list = [open_table(i,gama_dtypes,gama_date,gama_cols) for i in gama_files]

# Concatenate the list of dataframes into one dataframe.
print('Concatenating gama results: \n')
gama_results = pd.concat(gama_results_list)

# Dictionary to rename gama columns to match edf_results.
gama_to_edf_dict = {
    'GM_WELL_ID' : 'WID',
    'GM_CHEMICAL_VVL' : 'PARLABEL',
    'GM_RESULT_MODIFIER' : 'PARVQ',
    'GM_RESULT' : 'PARVAL',
    'GM_RESULT_UNITS' : 'UNITS',
    'GM_REPORTING_LIMIT' : 'REPDL',
    'GM_SAMP_COLLECTION_DATE' : 'LOGDATE',
}

# Rename gama columns to match edf_results.
gama_results = gama_results.rename(columns=gama_to_edf_dict)

gama_results['WID'] = gama_results['WID'].str.replace(' ','')

Loading GAMA results: 

Concatenating gama results: 



In [144]:
# Concatenate gama_results and edf_results.
samples = pd.concat([edf_results, gama_results])

# List of columns that require a value.
samples_req_cols = ['WID','LOGDATE', 'PARLABEL', 'PARVAL']

# Drops rows with missing values in required columns.
samples = samples.dropna(subset=samples_req_cols)

# Set multi index on WID and LOGDATE.
#samples = samples.set_index(['WID', 'LOGDATE']).sort_index()
samples['LOGDATE'] = samples['LOGDATE'].astype(str)
samples['GID'] = list(zip(samples['WID'], samples['LOGDATE']))
samples = samples.reset_index(drop=True)

### Location Data

In [145]:
# LOAD GEO XY PATH
geo_xy_path = bp / 'geotracker_xy'


def create_geo_xy(p):  # simple function for loading gama tables

    try:

        df = pd.read_csv(p, sep='\t', lineterminator='\n', encoding='unicode_escape',
                            quotechar='"',  quoting=3,  on_bad_lines='warn')

        df['WID'] = df['GLOBAL_ID'] + '-' + df['FIELD_PT_NAME']
        columns = ['WID', 'LATITUDE', 'LONGITUDE']
        df = df[columns]

        return df

    except:
        print('Exception, no such file.')


def concat_geo_xy(files):  # function to concat gama result datasets

    df_list = []

    for i in files:
        j = create_geo_xy(i)
        if j is not None:
            df_list.append(j)

    concatDF = pd.concat(df_list, axis=0)

    for df in df_list:
        del df

    return concatDF


geo_xy_files = geo_xy_path.glob('**/*.zip')
print('Loading Geotracker XY \n')
geo_xy = concat_geo_xy(geo_xy_files)

# geo_xy_gpd = gpd.GeoDataFrame(geo_xy, geometry=gpd.points_from_xy(geo_xy.LONGITUDE, geo_xy.LATITUDE), crs='EPSG:4326')

# load GAMA XY
print('Loading GAMA XY \n')
gama_xy_path = bp / "gama_xy\gama_location_construction_v2.zip"
gama_xy = pd.read_table(gama_xy_path, sep='\t', encoding='unicode_escape')

gama_xy.rename(columns={'GM_WELL_ID': 'WID', 'GM_LATITUDE': 'LATITUDE',
                'GM_LONGITUDE': 'LONGITUDE'}, inplace=True)
gama_xy_columns = ['WID', 'LATITUDE', 'LONGITUDE']
gama_xy = gama_xy[gama_xy_columns]

# combine well location data into singular dataset
print('Combining GAMA and Geotracker XY \n')
wells = pd.concat([gama_xy, geo_xy], ignore_index=True)
wells = wells.drop_duplicates(subset='WID').dropna(subset=['LATITUDE', 'LONGITUDE'])

Loading Geotracker XY 



b'Skipping line 18923: expected 13 fields, saw 14\nSkipping line 19016: expected 13 fields, saw 14\nSkipping line 27652: expected 13 fields, saw 14\nSkipping line 27653: expected 13 fields, saw 14\nSkipping line 27654: expected 13 fields, saw 14\nSkipping line 27823: expected 13 fields, saw 14\nSkipping line 27824: expected 13 fields, saw 14\nSkipping line 27825: expected 13 fields, saw 14\nSkipping line 29300: expected 13 fields, saw 14\nSkipping line 29301: expected 13 fields, saw 14\nSkipping line 29452: expected 13 fields, saw 14\nSkipping line 29453: expected 13 fields, saw 14\nSkipping line 29454: expected 13 fields, saw 14\nSkipping line 29455: expected 13 fields, saw 14\nSkipping line 29775: expected 13 fields, saw 14\nSkipping line 30039: expected 13 fields, saw 14\nSkipping line 30280: expected 13 fields, saw 14\nSkipping line 30281: expected 13 fields, saw 14\nSkipping line 32422: expected 13 fields, saw 14\nSkipping line 32423: expected 13 fields, saw 14\nSkipping line 3242

Loading GAMA XY 



c:\Users\chief\.conda\envs\geoprj\lib\site-packages\IPython\core\interactiveshell.py:3251: DtypeWarning: Columns (6,7,8) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


Combining GAMA and Geotracker XY 



### Unit Conversion Data

In [146]:
# Load conversion tables.
metric_conversion = pd.read_excel(bp / 'unit_conversion.xlsx', sheet_name='metric')
molar_conversion = pd.read_excel(bp / 'unit_conversion.xlsx', sheet_name='molar')

# join coversion factors to samples based on sample unit.
samples = samples.merge(metric_conversion, how='inner', left_on='UNITS', right_on='start_unit')

## **All Samples**

### Join MCL Table to Sample Results

In [147]:
# Select samples of selected contaminants 
samples = samples[samples['PARLABEL'].isin(contaminants_3)]

print('Loading MCL table \n')

# Create path to mcl table.
mcl_path = bp / 'MCL_list_1.xlsx'

# Open mcl table.
mcl = pd.read_excel(mcl_path,sheet_name='MCL', engine='openpyxl')

# join MCL values to sample results
print('Joining MCL values to samples \n')
samples_mcl = samples.merge(mcl, left_on='PARLABEL', right_on='chem_abrv', how='left')

Loading MCL table 

Joining MCL values to samples 



### Convert Units

In [148]:
# Create mask for samples with MCL units in UG/L and converts sample result units to UG/L.
mask = samples_mcl['units'] == 'UG/L'

# Multiply sample results by conversion factor.
samples_mcl.loc[mask, 'PARVAL'] = samples_mcl['PARVAL'] * samples_mcl['coef']

In [149]:
# Join well location data to sample results.
sample_results = samples_mcl.merge(wells, left_on='WID', right_on='WID', how='inner')
sample_results.drop(columns=['GID', 'start_unit', 'coef', 'UNITS', 'GID'], inplace=True)

### Select Specific Wells

In [150]:
sample_results = sample_results.loc[sample_results['LOGDATE'] >= '2010-01-01']

In [151]:
sample_groups = sample_results.groupby(['WID'])['PARLABEL'].apply(list).reset_index()

In [152]:
from collections import Counter


def select_wells(row):
    vals = row.values[1]
    counter = Counter(vals)
    print(counter)
    if len(counter) == len(contaminants_3):
        if all(values >= 4 for values in counter.values()) == True:
            print('True')
            print(row.values[0])
            print(counter.values())
            return True
        else:
            return False
    else:
        return False


res = sample_groups.apply(select_wells, axis=1)

Counter({'TCPR123': 2, 'BZME': 1, 'BZ': 1, 'EBZ': 1, 'PCE': 1, 'TCE': 1, 'XYLENES': 1, 'EDB': 1, 'MTBE': 1, 'DBCP': 1})
Counter({'TCPR123': 3, 'BZME': 2, 'BZ': 2, 'EBZ': 2, 'PCE': 2, 'TCE': 2, 'XYLENES': 2, 'EDB': 2, 'MTBE': 2, 'DBCP': 2})
Counter({'BZME': 1, 'BZ': 1, 'EBZ': 1, 'PCE': 1, 'TCE': 1, 'XYLENES': 1, 'TCPR123': 1, 'EDB': 1, 'MTBE': 1, 'DBCP': 1})
Counter({'BZME': 1, 'BZ': 1, 'EBZ': 1, 'PCE': 1, 'TCE': 1, 'XYLENES': 1, 'TCPR123': 1, 'EDB': 1, 'MTBE': 1, 'DBCP': 1})
Counter({'XYLENES': 60, 'BZ': 57, 'EBZ': 55, 'BZME': 55, 'MTBE': 11})
Counter({'BZME': 26, 'BZ': 26, 'XYLENES': 26, 'EBZ': 26})
Counter({'XYLENES': 26, 'BZME': 26, 'BZ': 26, 'EBZ': 26})
Counter({'BZ': 32, 'EBZ': 32, 'BZME': 32, 'XYLENES': 30})
Counter({'BZME': 20, 'EBZ': 20, 'XYLENES': 20, 'BZ': 20})
Counter({'BZME': 20, 'XYLENES': 20, 'EBZ': 20, 'BZ': 20})
Counter({'XYLENES': 22, 'BZME': 22, 'EBZ': 22, 'BZ': 22})
Counter({'BZ': 12, 'EBZ': 12, 'XYLENES': 12, 'BZME': 12})
Counter({'BZME': 14, 'EBZ': 14, 'BZ': 14, 'X

In [153]:
select_samples_results = sample_results[sample_results['WID'].isin(sample_groups.loc[res, 'WID'])]

In [154]:
# Create exceedence attribute, true if sample result exceeds reporting limit.
select_samples_results['exceedence'] = select_samples_results['PARVAL'] > select_samples_results['comp_conc_val']

# Create magnitude attribute. Sample result value divided by the comparison concentration value (MCL or Action level) minus 1.
select_samples_results['magnitude'] = (select_samples_results['PARVAL'] / select_samples_results['comp_conc_val']) - 1

C:\Users\chief\AppData\Local\Temp\ipykernel_16208\454334052.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  select_samples_results['exceedence'] = select_samples_results['PARVAL'] > select_samples_results['comp_conc_val']
C:\Users\chief\AppData\Local\Temp\ipykernel_16208\454334052.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  select_samples_results['magnitude'] = (select_samples_results['PARVAL'] / select_samples_results['comp_conc_val']) - 1


In [155]:
nwells = len(select_samples_results['WID'].unique())
print('Number of wells: ' + str(nwells))

Number of wells: 317


In [156]:
# Save sample results to csv.
select_samples_results.to_csv(results_path / '{}_spec_sample_results_10.csv'.format(county.lower()))

### run from

In [ ]:
#here

### Pivot table for CCME Water Quality Index

In [ ]:
sample_results.rename(columns={'WID' : 'Station', 'LOGDATE' : 'Date'}, inplace=True)

sample_results['PARLABEL'] = sample_results['PARLABEL'] + '_' + sample_results['units']

In [ ]:
pivot_table = pd.pivot_table(sample_results, index=['Station', 'Date'], columns=['PARLABEL'], values=['PARVAL'])

In [ ]:
ccme_wqi_data = pivot_table.reset_index()

In [ ]:
ccme_wqi_data.columns = ['Station', 'Date', 'AS_UG/L', 'BZME_UG/L', 'BZ_UG/L', 'CD_UG/L', 'DBCP_UG/L',
       'EBZ_UG/L', 'EDB_UG/L', 'MTBE_UG/L', 'NO3N_MG/L', 'PB_UG/L', 'PCE_UG/L',
       'TCE_UG/L', 'TCPR123_UG/L', 'THM_UG/L', 'XYLENES_UG/L']

In [ ]:
ccme_wqi_data.dropna(inplace=True)

In [ ]:
ccme_wqi_data

In [ ]:
ccme_wqi_data.to_csv(results_path / '{}_ccme_wqi_conc_samples.csv'.format(county.lower()))

### Normalize Sample Result Values at Wells

In [ ]:
# Calculates the mean of magnitudes for each WID in the exceedences dataframe.
print('Calculating magnitudes for each WID \n')
print(samples_mcl.head())

means = samples_mcl.groupby(['WID'])['magnitude'].mean()

In [ ]:
# Join mean magnitudes to well locations.
print('Merging geometric mean magnitudes to wells \n')
wells = wells.merge(means, how='inner', left_on='WID', right_index=True)
wells = wells.set_index('WID').sort_index()

# Save well mean magnitudes to csv.
wells.to_csv(bp / 'wells.csv')

In [ ]:
# Convert well mean magnitudes to shapefile
import geopandas as gpd

# Create geodataframe from well mean magnitudes, uses long and lat columns as xy coordinates, NAD83 projection.
gdf = gpd.GeoDataFrame(wells, geometry=gpd.points_from_xy(x=wells.LONGITUDE, y=wells.LATITUDE), crs='EPSG:4326')

# Reproject to UTM 11N.
gdf = gdf.to_crs('EPSG:26911')


gdf.to_file(results_path / 'wells.shp'.format(county))

## **Sample Groups**

### Create Sample Groups

In [ ]:
# Group samples by WID and LOGDATE apply list function to get list of PARLABELS for each group.
sample_groups = samples_mcl.groupby(['WID', 'LOGDATE'])['PARLABEL'].apply(list)

### Single Contaminant List

In [ ]:
# Use list comprehension to create a list of sample indexes where all contaminants in the contaminant list are present.
index_list = [i for i in sample_groups.index if all(item in sample_groups.loc[i] for item in contaminants_3)]

# Uses index_list to create a dataframe of samples that meet the criteria.
sample_group_results = samples_mcl[samples_mcl['GID'].isin(index_list)]

In [ ]:
# Print groups of samples that meet the criteria.
print('Groups: ',len(index_list))
print('Samples: ',len(sample_results))

In [ ]:
# Join location data to sample results.
sample_group_results = sample_results.merge(wells, left_on='WID', right_on='WID', how='inner')

# Save sample group results to csv.
sample_group_results.to_csv(bp / '{}_sample_results.csv'.format(county.lower()))

### Join Groundwater Elevations to Sample Results

In [157]:
# Create elev_path.
elev_path = bp / 'elevation'
print(elev_path, '\n')

# Dictionary of data types for gama_elev gama_elev for open_table().
gama_elev_dtypes = {
    'WELL NUMBER' : 'string',
    'DEPTH TO WATER' : 'float64',
    }

# Date column of gama_elev gama_elev for open_table().
gama_elev_date = ['MEASUREMENT DATE']

# Columns of gama_elev gama_elev for open_table().
gama_elev_cols = list(gama_elev_dtypes.keys()) + gama_elev_date


print('Loading GAMA groundwater elevations. \n')

# create list of files to open
gama_elev_files = elev_path.glob('**/*gama*.zip')
gama_elev_files = list(gama_elev_files)

# Use list comprehension to create a list of dataframes from the files list. Uses open_table() to open the files.
gama_elev_list = [open_table(i,gama_elev_dtypes,gama_elev_date,gama_elev_cols) for i in gama_elev_files]
print(gama_elev_list)

# Concatenate the list of dataframes into one dataframe if there are more than one.
if len(gama_elev_list) > 1:
    gama_elev = pd.concat(gama_elev_list)

else:
    gama_elev = gama_elev_list[0]

gama_geo_dict = {
    'WELL NUMBER' : 'WID',
    'DEPTH TO WATER' : 'DTW',
    'MEASUREMENT DATE' : 'LOGDATE',
}

gama_elev = gama_elev.rename(columns=gama_geo_dict)

gama_elev['LOGDATE'] = gama_elev['LOGDATE'].astype(str)

gama_elev['WID'] = gama_elev['WID'].str.replace(' ', '')

gama_elev['GID'] = list(zip(gama_elev['WID'], gama_elev['LOGDATE']))

e:\work\projects\coast_slr\scripts\ground_water_quality\elevation 

Loading GAMA groundwater elevations. 

[                  WELL NUMBER MEASUREMENT DATE  DEPTH TO WATER
0               17S11E22E002S       1993-10-19          100.52
1               17S11E22E002S       1994-03-14          100.41
2               17S11E22E002S       1994-10-17          100.43
3               17S11E22E002S       1995-03-29          100.36
4               17S11E22E002S       1995-10-16          100.17
...                       ...              ...             ...
4229858  SL204131495 - REW-12       2021-11-16           72.20
4229859  SL204131495 - REW-12       2021-12-07           72.33
4229860  SL204131495 - REW-14       2017-04-04             NaN
4229861  SL204131495 - REW-15       2017-04-04             NaN
4229862   SL204131495 - REW-3       2017-04-04             NaN

[4229863 rows x 3 columns]]


In [158]:
# Dictionary of data types for geo_elev geo_elev for open_table().
geo_elev_dtypes = {
    'GLOBAL_ID' : 'string',
    'FIELD_POINT_NAME' : 'string',
    'DTW' : 'float64',
    }

# Date column of geo_elev geo_elev for open_table().
geo_elev_date = ['GW_MEAS_DATE']

# Columns of geo_elev geo_elev for open_table().
geo_elev_cols = list(geo_elev_dtypes.keys()) + geo_elev_date

print('Loading Geotracker groundwater elevations. \n')

# create list of files to open
geo_elev_files = elev_path.glob('**/*Geo*.zip')
geo_elev_files = list(geo_elev_files)


# Use list comprehension to create a list of dataframes from the files list. Uses open_table() to open the files.
geo_elev_list = [open_table(i,geo_elev_dtypes,geo_elev_date,geo_elev_cols) for i in geo_elev_files]



# Concatenate the list of dataframes into one dataframe if there are more than one.
if len(geo_elev_list) > 1:
    geo_elev = pd.concat(geo_elev_list)

else:
    geo_elev = geo_elev_list[0]

# Create WID column.
geo_elev['WID'] = geo_elev['GLOBAL_ID'] + '-' + geo_elev['FIELD_POINT_NAME']

geo_elev['WID'] = geo_elev['WID'].str.replace(' ', '')

# Drop unnecessary columns.
geo_elev = geo_elev.drop(columns=['GLOBAL_ID', 'FIELD_POINT_NAME'])

geo_elev = geo_elev.rename(columns={'GW_MEAS_DATE' : 'LOGDATE'})

geo_elev['LOGDATE'] = geo_elev['LOGDATE'].astype(str)

geo_elev['GID'] = list(zip(geo_elev['WID'], geo_elev['LOGDATE']))

Loading Geotracker groundwater elevations. 



In [166]:
select_samples_results

,LOGDATE,PARLABEL,PARVAL,PARVQ,REPDL,WID,chem_abrv,chem_name,units,comp_conc_val,comp_conc_type,LATITUDE,LONGITUDE,exceedence,magnitude,GID
5805,2018-06-25,TCPR123,0.0,ND,1.0,GAOG10010391-32SE-EB-1,TCPR123,"1,2,3-Trichloropropane\n(1,2,3 TCP)",UG/L,0.005,MCL,35.619265,-119.729334,False,-1.0,"(GAOG10010391-32SE-EB-1, 2018-06-25)"
5806,2018-06-25,PCE,0.0,ND,0.5,GAOG10010391-32SE-EB-1,PCE,Tetrachloroethene (PCE),UG/L,5.000,MCL,35.619265,-119.729334,False,-1.0,"(GAOG10010391-32SE-EB-1, 2018-06-25)"
5807,2021-03-10,BZ,9.8,=,0.5,GAOG10010391-32SE-EB-1,BZ,Benzene,UG/L,1.000,MCL,35.619265,-119.729334,True,8.8,"(GAOG10010391-32SE-EB-1, 2021-03-10)"
5808,2018-06-25,BZME,0.0,ND,0.5,GAOG10010391-32SE-EB-1,BZME,Toluene,UG/L,150.000,MCL,35.619265,-119.729334,False,-1.0,"(GAOG10010391-32SE-EB-1, 2018-06-25)"
5809,2019-03-18,TCPR123,0.0,ND,1.0,GAOG10010391-32SE-EB-1,TCPR123,"1,2,3-Trichloropropane\n(1,2,3 TCP)",UG/L,0.005,MCL,35.619265,-119.729334,False,-1.0,"(GAOG10010391-32SE-EB-1, 2019-03-18)"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
266174,2021-04-05,EDB,0.0,ND,1.0,SL0603728988-MW-8,EDB,"1,2 Dibromoethane (EDB)",UG/L,0.050,MCL,35.213535,-118.246613,False,-1.0,"(SL0603728988-MW-8, 2021-04-05)"
266175,2019-10-28,XYLENES,0.0,ND,3.0,SL0603728988-MW-8,XYLENES,Xylenes (total),UG/L,1750.000,MCL,35.213535,-118.246613,False,-1.0,"(SL0603728988-MW-8, 2019-10-28)"
266176,2019-04-02,XYLENES,0.0,ND,3.0,SL0603728988-MW-8,XYLENES,Xylenes (total),UG/L,1750.000,MCL,35.213535,-118.246613,False,-1.0,"(SL0603728988-MW-8, 2019-04-02)"
266177,2019-04-02,DBCP,0.0,ND,5.0,SL0603728988-MW-8,DBCP,"1,2-Dibromo-3-chloropropane (DBCP)",UG/L,0.200,MCL,35.213535,-118.246613,False,-1.0,"(SL0603728988-MW-8, 2019-04-02)"


In [159]:
# Concatenate gama_results and edf_results.
dtw = pd.concat([geo_elev, gama_elev])

# List of columns that require a value.
dtw_req_cols = ['WID','DTW','LOGDATE']

# Drops rows with missing values in required columns.
dtw = dtw.dropna(subset=dtw_req_cols)

# Set multi index on WID and LOGDATE.
#dtw = dtw.set_index(['WID', 'LOGDATE']).sort_index()

In [160]:
dtw = dtw.drop_duplicates(subset=['GID'])

In [161]:
select_samples_results['GID'] = list(zip(select_samples_results['WID'], select_samples_results['LOGDATE']))

C:\Users\chief\AppData\Local\Temp\ipykernel_16208\3634736721.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  select_samples_results['GID'] = list(zip(select_samples_results['WID'], select_samples_results['LOGDATE']))


In [162]:
len(select_samples_results['WID'].unique())

317

In [163]:
select_samples_results[select_samples_results['GID'].isin(dtw['GID'])]

,LOGDATE,PARLABEL,PARVAL,PARVQ,REPDL,WID,chem_abrv,chem_name,units,comp_conc_val,comp_conc_type,LATITUDE,LONGITUDE,exceedence,magnitude,GID
5805,2018-06-25,TCPR123,0.0,ND,1.0,GAOG10010391-32SE-EB-1,TCPR123,"1,2,3-Trichloropropane\n(1,2,3 TCP)",UG/L,0.005,MCL,35.619265,-119.729334,False,-1.0,"(GAOG10010391-32SE-EB-1, 2018-06-25)"
5806,2018-06-25,PCE,0.0,ND,0.5,GAOG10010391-32SE-EB-1,PCE,Tetrachloroethene (PCE),UG/L,5.000,MCL,35.619265,-119.729334,False,-1.0,"(GAOG10010391-32SE-EB-1, 2018-06-25)"
5807,2021-03-10,BZ,9.8,=,0.5,GAOG10010391-32SE-EB-1,BZ,Benzene,UG/L,1.000,MCL,35.619265,-119.729334,True,8.8,"(GAOG10010391-32SE-EB-1, 2021-03-10)"
5808,2018-06-25,BZME,0.0,ND,0.5,GAOG10010391-32SE-EB-1,BZME,Toluene,UG/L,150.000,MCL,35.619265,-119.729334,False,-1.0,"(GAOG10010391-32SE-EB-1, 2018-06-25)"
5809,2019-03-18,TCPR123,0.0,ND,1.0,GAOG10010391-32SE-EB-1,TCPR123,"1,2,3-Trichloropropane\n(1,2,3 TCP)",UG/L,0.005,MCL,35.619265,-119.729334,False,-1.0,"(GAOG10010391-32SE-EB-1, 2019-03-18)"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
266174,2021-04-05,EDB,0.0,ND,1.0,SL0603728988-MW-8,EDB,"1,2 Dibromoethane (EDB)",UG/L,0.050,MCL,35.213535,-118.246613,False,-1.0,"(SL0603728988-MW-8, 2021-04-05)"
266175,2019-10-28,XYLENES,0.0,ND,3.0,SL0603728988-MW-8,XYLENES,Xylenes (total),UG/L,1750.000,MCL,35.213535,-118.246613,False,-1.0,"(SL0603728988-MW-8, 2019-10-28)"
266176,2019-04-02,XYLENES,0.0,ND,3.0,SL0603728988-MW-8,XYLENES,Xylenes (total),UG/L,1750.000,MCL,35.213535,-118.246613,False,-1.0,"(SL0603728988-MW-8, 2019-04-02)"
266177,2019-04-02,DBCP,0.0,ND,5.0,SL0603728988-MW-8,DBCP,"1,2-Dibromo-3-chloropropane (DBCP)",UG/L,0.200,MCL,35.213535,-118.246613,False,-1.0,"(SL0603728988-MW-8, 2019-04-02)"


In [164]:
select_samples_results_dtw = select_samples_results.merge(dtw, left_on=['GID'], right_on=['GID'], how='inner')

In [108]:
select_samples_results_dtw

,LOGDATE_x,PARLABEL,PARVAL,PARVQ,REPDL,WID_x,chem_abrv,chem_name,units,comp_conc_val,comp_conc_type,LATITUDE,LONGITUDE,exceedence,magnitude,GID,LOGDATE_y,DTW,WID_y
0,2016-09-06,PCE,0.0,ND,5.0,L10001396701-E-22,PCE,Tetrachloroethene (PCE),UG/L,5.00,MCL,34.292070,-118.802944,False,-1.0,"(L10001396701-E-22, 2016-09-06)",2016-09-06,128.51,L10001396701-E-22
1,2016-09-06,TCE,0.0,ND,5.0,L10001396701-E-22,TCE,Trichloroethene (TCE),UG/L,5.00,MCL,34.292070,-118.802944,False,-1.0,"(L10001396701-E-22, 2016-09-06)",2016-09-06,128.51,L10001396701-E-22
2,2016-09-06,MTBE,0.0,ND,5.0,L10001396701-E-22,MTBE,MTBE (Methyl-tert-butyl\nether),UG/L,13.00,MCL,34.292070,-118.802944,False,-1.0,"(L10001396701-E-22, 2016-09-06)",2016-09-06,128.51,L10001396701-E-22
3,2016-09-06,EDB,0.0,ND,1.0,L10001396701-E-22,EDB,"1,2 Dibromoethane (EDB)",UG/L,0.05,MCL,34.292070,-118.802944,False,-1.0,"(L10001396701-E-22, 2016-09-06)",2016-09-06,128.51,L10001396701-E-22
4,2016-09-06,PCE,0.0,ND,5.0,L10001396701-E-22,PCE,Tetrachloroethene (PCE),UG/L,5.00,MCL,34.292070,-118.802944,False,-1.0,"(L10001396701-E-22, 2016-09-06)",2016-09-06,128.51,L10001396701-E-22
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
35680,2011-08-04,DBCP,2.0,ND,2.0,T0611100108-MW-48,DBCP,"1,2-Dibromo-3-chloropropane (DBCP)",UG/L,0.20,MCL,34.119773,-119.102807,True,9.0,"(T0611100108-MW-48, 2011-08-04)",2011-08-04,4.46,T0611100108-MW-48
35681,2010-07-14,BZ,1.0,ND,1.0,T0611100108-MW-48,BZ,Benzene,UG/L,1.00,MCL,34.119773,-119.102807,False,0.0,"(T0611100108-MW-48, 2010-07-14)",2010-07-14,4.70,T0611100108-MW-48
35682,2010-07-14,MTBE,2.0,ND,2.0,T0611100108-MW-48,MTBE,MTBE (Methyl-tert-butyl\nether),UG/L,13.00,MCL,34.119773,-119.102807,False,-0.846154,"(T0611100108-MW-48, 2010-07-14)",2010-07-14,4.70,T0611100108-MW-48
35683,2010-07-14,BZ,1.0,ND,1.0,T0611100108-MW-48,BZ,Benzene,UG/L,1.00,MCL,34.119773,-119.102807,False,0.0,"(T0611100108-MW-48, 2010-07-14)",2010-07-14,4.70,T0611100108-MW-48


In [165]:
a = (len(select_samples_results))
b = (len(select_samples_results_dtw))
c =((len(select_samples_results_dtw) / len(select_samples_results)*100))

print('There are ' + str(b) + ' samples with depth to water values.')
print("Out of " + str(a) + " samples in the original dataframe.")
print("That's " + str(c) + "% of the samples. \n")

a = (len(select_samples_results['WID'].unique()))
b = (len(select_samples_results_dtw['WID_x'].unique()))
c = ((len(select_samples_results_dtw['WID_x'].unique()) / len(select_samples_results['WID'].unique())*100))

print('There are ' + str(b) + ' wells with depth to water values.')
print("Out of " + str(a) + " wells in the original dataframe.")
print("That's " + str(c) + "% of the wells. \n")

There are 7908 samples with depth to water values.
Out of 92491 samples in the original dataframe.
That's 8.550021083132412% of the samples. 

There are 67 wells with depth to water values.
Out of 317 wells in the original dataframe.
That's 21.13564668769716% of the wells. 



In [ ]:
sample_results.to_csv(bp / '{}_sample_results.csv'.format(county.lower()))

### Contaminant Combinations

In [ ]:
from itertools import combinations

# Create list of all combinations of all contaminants
combinations_list = list(combinations(contaminants_3, 10))
len(combinations_list)

In [ ]:
# Function to select sample groups based on combinations of contaminants.
def get_select_samples(row, contaminants):

    # checks list of contaminants in row against list of contaminants in function call.
    # if all contaminants in row are in contaminants, return True.
    if all(item in row for item in contaminants):
        return True

    else:
        return False

In [ ]:
def get_select_samples(row, contaminants):

    count = 0

    for values in row:
        print(values)
        if all(item in values for item in contaminants):
            count += 1
    return count

In [ ]:
ser_dict = {}

total =  len(combinations_list)
count = 0

for contaminants in combinations_list:

    count += 1
    percent = int(((count/total)*100))

    ser = sample_groups.apply(get_select_samples, contaminants=contaminants_3)

    ser_dict[contaminants] = ser

    #print('{}%'.format(percent))
    
combo_stats = pd.DataFrame.from_dict(ser_dict, orient='index')

print(combo_stats.max())
print(list(combo_stats.idxmax()))

In [ ]:
ser_dict

## **Modin Combo Stats**

In [ ]:
def get_select_samples_modin(row, contaminants):
    print(row)

    if all(element in row for element in contaminants) ==  True:
        print('contains all elements')
        return True

    else:
        print('does not contain all elements')
        return False

In [ ]:
combinations_list

In [ ]:
samples_modin = mpd.DataFrame(samples)

In [ ]:
import modin.pandas as mpd
from distributed import Client
client = Client()

sample_groups_modin = mpd.DataFrame(sample_groups)


ser_dict = {}

for contaminants in combinations_list:

    ser = sample_groups_modin.apply(get_select_samples_modin, contaminants=contaminants)
    ser = ser[ser == True]

    ser_dict[contaminants] = len(ser)



print(max(ser_dict.values()))

In [ ]:
print(combo_stats.max())
print(list(combo_stats.idxmax()))

In [ ]:
df[df == True]

In [ ]:
df[df == True]

In [ ]:
print('Loading MCL table \n')

# Create path to mcl table.
mcl_path = bp / 'MCL_list_1.xlsx'

# Open mcl table.
mcl = pd.read_excel(mcl_path, engine='openpyxl')

# join MCL values to sample results
print('Joining MCL values to samples \n')
samples_mcl = select_samples.merge(mcl, left_on='PARLABEL', right_on='chem_abrv', how='left').set_index(select_samples.index)

In [ ]:
# Save samples_mcl to csv.
alt = input("Input filename ending for 'county'_select_samples_'input'.csv: ")
name = '{}_select_samples_{}.csv'.format(county.lower(), alt)
sp = bp / name
samples_mcl.to_csv(sp)

In [ ]:
# Get counts of samples for each contaminant.
parlabel_stats = samples['PARLABEL'].value_counts()

# Create a dataframe with the counts of samples for each contaminant.
parlabel_stats = parlabel_stats.to_frame(name='COUNTS').reset_index().rename(columns={'index':'PARLABEL'})

# Create PERCENT column for each contaminant. Showing percent of samples for each contaminant compared to total samples.
parlabel_stats['PERCENT'] = (parlabel_stats['COUNTS'] / len(samples) * 100).round(4)

In [ ]:
# Save samples_mcl to csv.
name = '{}_parlabel_stats.csv'.format(county.lower())
sp = bp / name
parlabel_stats.to_csv(sp)